In [ ]:
#31st Dec'24

In [ ]:
#We should convert text into vectors after splitting the overall data into train and test datsets, this will prevent data leakage.

In [ ]:
!pip install gensim --quiet

In [1]:
from gensim.models import Word2Vec, KeyedVectors

In [2]:
#import gensim.downloader as api
#wv = api.load('word2vec-google-news-300')

In [3]:
import pandas as pd
data = pd.read_csv("/content/SMSSpamCollection.txt", sep="\t",names=['label','message'])
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.shape

(5572, 2)

In [5]:
data.index

RangeIndex(start=0, stop=5572, step=1)

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
lemmatizer = WordNetLemmatizer()

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [40]:
words = []
for i in range(0,len(data)):

  sent_token = simple_preprocess(data["message"][i])
  #sent_token = [lemmatizer.lemmatize(word) for word in sent_token]
  #print(f"ctr= {ctr} Sent token = {sent_token}")
  #for sent in (sent_token):
    #print(f"Simple preprocess = {simple_preprocess(i)}\n")
  words.append([lemmatizer.lemmatize(word) for word in sent_token])


In [44]:
data["preprocessed"] = words
data.head()

,label,message,preprocessed
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, wkly, comp, to, win, fa, cup..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, don, think, he, go, to, usf, he, life, a..."


In [45]:
import gensim
model = gensim.models.Word2Vec(sentences=data["preprocessed"])

In [48]:
#Getting all the vocab
#model.wv.index_to_key

1727

In [49]:
model.corpus_count

5572

In [50]:
model.epochs

5

In [51]:
model.wv.similar_by_word("kid")

[('many', 0.9969536066055298),
 ('finish', 0.9968774914741516),
 ('those', 0.9968673586845398),
 ('really', 0.9968559145927429),
 ('friend', 0.9968552589416504),
 ('yup', 0.996805727481842),
 ('even', 0.996802568435669),
 ('amp', 0.9968003034591675),
 ('dear', 0.9967779517173767),
 ('sweet', 0.9967694282531738)]

In [52]:
model.wv["good"].shape

(100,)

In [54]:
#words[0]

In [55]:
!pip install tqdm
from tqdm import tqdm
import numpy as np

In [56]:
def avg_word2vec(doc):
  # Get word vectors for words present in the vocabulary
  vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]

  # Handle cases where no words are found in the vocabulary
  if not vectors:  # If vectors are empty
    return np.zeros(model.vector_size)  # Return a zero vector of the appropriate size
  else:
    return np.mean(vectors, axis=0)  # Calculate the average


data["vector"] = data["preprocessed"].apply(lambda X: avg_word2vec(X))


In [77]:
X = data["vector"]

In [78]:
X[0]

array([-7.09360465e-02,  2.93344200e-01,  1.46740004e-01,  2.26620771e-02,
        9.69144404e-02, -3.80087852e-01,  1.40718907e-01,  5.20283997e-01,
       -2.14607865e-01, -1.31871343e-01, -1.87222049e-01, -3.79549414e-01,
       -6.05735071e-02,  4.89736944e-02,  1.12669870e-01, -2.03744948e-01,
        3.48653607e-02, -2.75834620e-01,  3.42763902e-04, -5.56740403e-01,
        2.06811965e-01,  1.61191672e-01,  2.06175864e-01, -1.71568692e-01,
       -6.92091286e-02,  6.75445572e-02, -2.77959704e-01, -2.17005238e-01,
       -2.90510595e-01,  3.11076473e-02,  2.08158955e-01, -2.47264858e-02,
        1.93207100e-01, -2.39022672e-01, -1.17357478e-01,  3.25090140e-01,
        4.72385027e-02, -2.30715215e-01, -1.40994608e-01, -4.37252998e-01,
        1.07614391e-01, -2.31950611e-01, -1.66635647e-01,  1.10296406e-01,
        2.26483837e-01, -6.69241026e-02, -1.76252335e-01, -3.62877883e-02,
        2.00101107e-01,  1.51706427e-01,  2.08235055e-01, -2.33061850e-01,
       -2.07952168e-02, -

In [79]:
X_new = []
for i in X:
  X_new.append(i.reshape(-1))

X_new = np.array(X_new)


In [81]:
X_new.shape

(5572, 100)

In [ ]:
data["label"].value_counts()

,count
label,
ham,4825
spam,747


In [83]:
data["label"].value_counts()

,count
label,
ham,4825
spam,747


In [88]:
#Dependent Features
Y = data[["label"]].replace({"ham": 1, "spam": 0})

<ipython-input-88-2df908f4b07b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = data[["label"]].replace({"ham": 1, "spam": 0})


In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_new,Y,test_size=0.2,random_state=2)

In [91]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [92]:
model.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [93]:
Y_pred = model.predict(X_test)

In [94]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy_score(Y_test,Y_pred)

0.9605381165919282

In [95]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.90      0.82      0.85       158
           1       0.97      0.98      0.98       957

    accuracy                           0.96      1115
   macro avg       0.93      0.90      0.92      1115
weighted avg       0.96      0.96      0.96      1115



In [96]:
print(confusion_matrix(Y_test,Y_pred))

[[129  29]
 [ 15 942]]
